In [ ]:
# Imports and definitions

import keras
import numpy as np
from datetime import datetime

from os import listdir, makedirs
from os.path import exists, isfile, join

model_name = "board-and-lines"


In [ ]:
# Create model

position_shape = (2, 7, 6)
lines_shape = (4,)

position_input = keras.layers.Input(shape=position_shape, name="regression: board and lines")
flatten = keras.layers.Flatten()(position_input)
lines_input = keras.layers.Input(shape=lines_shape)
concatenated = keras.layers.Concatenate()([flatten, lines_input, lines_input])

dense1 = keras.layers.Dense(1024, activation = "relu")(concatenated)
dense2 = keras.layers.Dense(512, activation = "relu")(dense1)
dense3 = keras.layers.Dense(256, activation = "relu")(dense2)

output = keras.layers.Dense(1, activation = "linear", name = "output")(dense3)

model = keras.models.Model(inputs=[position_input, lines_input, lines_input], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()


In [ ]:
# Save model

if not exists(f"models/{model_name}"):
    makedirs(f"models/{model_name}")
model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")


In [ ]:
# Train model

positions = np.load("positions.npy")
evaluations = np.load("evaluations.npy")
player1_lines = np.load("player1_lines.npy")
player2_lines = np.load("player2_lines.npy")

while True:
    model_version = sorted([f for f in listdir(f"models/{model_name}") if isfile(join(f"models/{model_name}", f))])[-1]

    model = keras.models.load_model(f"models/{model_name}/{model_version}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
    model.fit([positions, player1_lines, player2_lines], evaluations, epochs = 1, batch_size = 128, validation_split = 0.2)

    model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")
